In [3]:
import pandas as pd 
import numpy as np

In [4]:
df = pd.read_csv(r'C:\Users\22213\OneDrive\Desktop\recani\recani\data and model\test_data_pre_processed.csv')

In [5]:
pd.set_option('display.max_columns', None)
display.max_characters = None

In [6]:
df.head(1)

,name_english,name,score,ranked,popularity,members,synopsis,synonyms,type_of,total_episodes,premiered,studios,genres,demographic,duration_per_ep,rating,scored_by,favorites,aired,source,watching,completed,on_hold,dropped,plan_to_watch,total,scored_10_by,scored_9_by,scored_8_by,scored_7_by,scored_6_by,scored_5_by,scored_4_by,scored_3_by,scored_2_by,scored_1_by
0,Tomorrow's Joe 2,Ashita no Joe 2,8.73,50,3041,51773,Yabuki Joe is left downhearted and hopeless af...,Rocky Joe 2,TV,47.0,Fall 1980,Tokyo Movie Shinsha,"['Drama', 'Historical', 'Sports', 'Adventure']",Shounen,24 min. per ep.,PG-13 - Teens 13 or older,20201.0,2653,"Oct 13, 1980 to Aug 31, 1981",Manga,2723,23933,1105,1205,22807,51773,7721.0,4711.0,3690.0,1797.0,653.0,297.0,120.0,137.0,182.0,893.0


In [7]:
df['demographic'] = df['demographic'].apply(lambda x: [x])

In [8]:
df.head(1)

,name_english,name,score,ranked,popularity,members,synopsis,synonyms,type_of,total_episodes,premiered,studios,genres,demographic,duration_per_ep,rating,scored_by,favorites,aired,source,watching,completed,on_hold,dropped,plan_to_watch,total,scored_10_by,scored_9_by,scored_8_by,scored_7_by,scored_6_by,scored_5_by,scored_4_by,scored_3_by,scored_2_by,scored_1_by
0,Tomorrow's Joe 2,Ashita no Joe 2,8.73,50,3041,51773,Yabuki Joe is left downhearted and hopeless af...,Rocky Joe 2,TV,47.0,Fall 1980,Tokyo Movie Shinsha,"['Drama', 'Historical', 'Sports', 'Adventure']",[Shounen],24 min. per ep.,PG-13 - Teens 13 or older,20201.0,2653,"Oct 13, 1980 to Aug 31, 1981",Manga,2723,23933,1105,1205,22807,51773,7721.0,4711.0,3690.0,1797.0,653.0,297.0,120.0,137.0,182.0,893.0


In [9]:
df['genres'][1]

"['Mystery', 'Supernatural', 'Action', 'Adventure', 'Slice of Life']"

In [10]:
type(df['genres'][18])

str

In [11]:
import ast

In [12]:
def convert_to_list_genres(x):
    if type(x) != float:
        return ', '.join(ast.literal_eval(x))
    else:
        return np.nan

In [13]:
df['genres'] = df['genres'].apply(convert_to_list_genres)

In [14]:
def convert_to_list_demographic(x):
    if not any(pd.isna(a) for a in x):
        return ', ' + ''.join(x)
    else:
        return ' '

In [15]:
df['demographic'] = df['demographic'].apply(convert_to_list_demographic)

In [16]:
df['genres'] = df['genres'].fillna('')
df['demographic'] = df['demographic'].fillna('')

In [17]:
df['input_string'] = df['synopsis'] + df['genres'] + df['demographic'] + '.'

In [18]:
type(df['input_string'][1])

str

In [19]:
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec

model = Word2Vec.load(r'C:\Users\22213\OneDrive\Desktop\recani\recani\data and model\Word2Vec_local.bin')

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    
    return words

def get_text_vector(text, model):
    # Preprocess the text
    processed_text = preprocess_text(text)
    
    # Initialize an empty vector
    text_vector = []
    
    # Iterate over each word in the processed text
    for word in processed_text:
        # Check if the word is in the vocabulary of the Word2Vec model
        if word in model.wv.key_to_index:
            # Get the word vector from the model and append it to the text vector
            text_vector.append(model.wv[word])
    
    # Calculate the mean of all word vectors to get the text vector
    if text_vector:
        text_vector = sum(text_vector) / len(text_vector)
    
    return text_vector

In [20]:
df['anime_vector'] = df['input_string'].apply(lambda x: get_text_vector(x, model))

In [21]:
df = df[['anime_vector', 'name_english']]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_vector  14740 non-null  object
 1   name_english  14740 non-null  object
dtypes: object(2)
memory usage: 230.4+ KB


In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
vectors = np.array(df['anime_vector'].tolist())

In [25]:
similarity_matrix = cosine_similarity(vectors, vectors)

In [26]:
similarity_matrix

array([[1.        , 0.9530309 , 0.9332725 , ..., 0.7786238 , 0.93567604,
        0.9628992 ],
       [0.9530309 , 1.        , 0.93315005, ..., 0.79451954, 0.92924505,
        0.96024066],
       [0.9332725 , 0.93315005, 1.0000002 , ..., 0.70951647, 0.8545269 ,
        0.92682874],
       ...,
       [0.7786238 , 0.79451954, 0.70951647, ..., 1.        , 0.8265307 ,
        0.80110204],
       [0.93567604, 0.92924505, 0.8545269 , ..., 0.8265307 , 1.0000001 ,
        0.9295829 ],
       [0.9628992 , 0.96024066, 0.92682874, ..., 0.80110204, 0.9295829 ,
        0.99999994]], dtype=float32)

In [27]:
def recommend(anime):
    anime_index = df[df['name_english'] == anime].index[0]
    distances = similarity_matrix[anime_index]
    anime_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x:x[1])[1:4]
    for i in anime_list:
        print(df.iloc[i[0]].name_english)

In [28]:
recommend("One Piece")

One Piece 3D2Y: Overcoming Ace's Death! Luffy's Pledge to His Friends
Shin Koihime†Musou
Berserk (2016)


In [30]:
import pickle

with open(r'C:\Users\22213\OneDrive\Desktop\recani\recani\data and model\similarity_matrix.pkl', 'wb') as f:
    pickle.dump(similarity_matrix, f)